In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import random

import cv2, os, shutil
from matplotlib import pyplot as plt

import albumentations as A

C:\Users\sba\miniconda3\envs\album\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White

def visualize_bbox(img, bbox, class_name, pic_w, pic_h, color=BOX_COLOR, thickness=2):
    dataType = "yolo"

    """Visualizes a single bounding box on the image"""

    if dataType == 'coco':
        x_min, y_min, w, h = bbox # 정규화 된 0~1 사이의 값
        x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    elif dataType == "yolo":
        x_center, y_center, w, h = bbox
        # 픽셀 좌표로 변환 이미지의 width와 height값
        x_min = int(float(x_center - w/2) * pic_w)
        x_max = int(float(x_center + w/2) * pic_w)
        y_min = int(float(y_center - h/2) * pic_h)
        y_max = int(float(y_center + h/2) * pic_h)
    print(w, h)
    print(x_min, y_min, y_min, y_max)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35,
        color=TEXT_COLOR,
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name, img_shape):
    img = image.copy()
    h,w = img_shape[0:2]
    print(bboxes)
    print(category_ids)
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name, w, h)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [4]:
import os
from glob import glob 

currDir = os.getcwd()
print(currDir)

# 데이터 증식 폴더 만들기
augFolder = 'kAug_multOverBatch'
augDataDir = os.path.join(currDir, augFolder)

if os.path.exists(augDataDir):
    shutil.rmtree(augDataDir)
os.mkdir(augDataDir)

# 원본 데이터셋이 저장된 폴더
orgDataPath = 'C:\\Users\\SBA\\yolo\\yolov11\\kOrig_multOver'
imagefiles = glob(os.path.join(orgDataPath, '*.JPG'))
labelfiles = glob(os.path.join(orgDataPath, '*.txt'))
# print(imagefiles)
# print(labelfiles)

C:\Users\SBA\yolo\yolov11


In [5]:
def read_label_txt(txtFile):
    category_ids = []
    bboxes = []

    f=open(txtFile,'r')

    while True:
        line = f.readline()
        if not line: break
        ids, xc, yc, w, h= line.split(' ')
        category_ids.append(int(ids))
        bboxes.append([float(xc),float(yc),float(w),float(h)])
        #print(line)
    f.close()
    return category_ids, bboxes

In [6]:
def write_label_txt(txtFile, category_ids, bboxes):
    f=open(txtFile,'w')

    for i, ids in enumerate(category_ids):
        xc,yc,w,h = bboxes[i]
        f.write("{} {} {} {} {}\n".format(int(ids),xc,yc,w,h))
        #print("{} {}".format(int(ids), bboxes[i]))
    f.close()

In [7]:
category_id_to_name = {0: 'kido'}

In [8]:
transform_resize = A.Compose(
    [A.LongestMaxSize(max_size=800, interpolation=cv2.INTER_LINEAR),],
    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']),
)

transform_hflip = A.Compose(
    [A.HorizontalFlip(p=1.0)],
    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']),
)

In [9]:
DEBUG = False

def dataAug(func, imagefiles, augDir):
    for imagefile in imagefiles:
        image = cv2.imread(imagefile)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_height, img_width = image.shape[0:2]
    
        # txt파일은 이미지 파일에서 확장자만 다르다!
        readTxtFile = imagefile.split('.')[0] + '.txt'
        baseName = os.path.basename(imagefile).split('.')[0]
        writeTxtFile = os.path.join(augDir, str(baseName+'.txt'))
        category_ids, bboxes = read_label_txt(readTxtFile)
        
        if func=='hflip':
            transformed = transform_hflip(image=image, bboxes=bboxes, category_ids=category_ids)
            saveImageName = baseName + '_hflip.JPG'  
            saveLabelName = baseName + '_hflip.txt'
            saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
            saveImgPath = os.path.join(augDir, saveImageName)
            saveLabelPath = os.path.join(augDir, saveLabelName)
            cv2.imwrite(saveImgPath, saveImage)
            write_label_txt(saveLabelPath, transformed['category_ids'], transformed['bboxes'])            
        elif func=='rotate':
            angle_inter = 20
            for angle in range(angle_inter,360,angle_inter):
                transform_rotate = A.Compose(
                    [A.Rotate(limit=(angle,angle), rotate_method='largest_box', p=1.0)],
                    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids'])
                )
                transformed = transform_rotate(image=image, bboxes=bboxes, category_ids=category_ids)
                saveImageName = baseName + '_rotate_{}.JPG'.format(angle)  
                saveLabelName = baseName + '_rotate_{}.txt'.format(angle)  
                saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
                saveImgPath = os.path.join(augDir, saveImageName)
                saveLabelPath = os.path.join(augDir, saveLabelName)
                cv2.imwrite(saveImgPath, saveImage)
                write_label_txt(saveLabelPath, transformed['category_ids'], transformed['bboxes'])
        elif func=='resize':
            transformed = transform_resize(image=image, bboxes=bboxes, category_ids=category_ids)
            saveImageName = baseName + '.JPG'  
            saveLabelName = baseName + '.txt'            
            saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
            saveImgPath = os.path.join(augDir, saveImageName)
            saveLabelPath = os.path.join(augDir, saveLabelName)
            cv2.imwrite(saveImgPath, saveImage)
            write_label_txt(saveLabelPath, transformed['category_ids'], transformed['bboxes'])
            
        if DEBUG==True:
            visualize(
                transformed['image'],
                transformed['bboxes'],
                transformed['category_ids'],
                category_id_to_name,
                img_width, img_height
            )
            break

In [10]:
dataAug('resize', imagefiles, augDataDir)
dataAug('hflip', imagefiles, augDataDir)
dataAug('rotate', imagefiles, augDataDir)

In [11]:
from ultralytics import YOLO
model = YOLO('yolo11n.pt')

In [12]:
# small데이터셋을 위한 폴더를 지정하고 4000장을 나눠서 보관
import os, shutil

sdata = os.path.join(os.getcwd(), 'kAug_multOverBatch_tvt')

if os.path.exists(sdata):
    shutil.rmtree(sdata)
os.makedirs(sdata)

train_sdata = os.path.join(sdata,'train')
valid_sdata = os.path.join(sdata,'valid')
test_sdata = os.path.join(sdata,'test')

trainValidTestList = ['train','valid','test']

os.makedirs(train_sdata, exist_ok=True)
os.makedirs(valid_sdata, exist_ok=True)
os.makedirs(test_sdata, exist_ok=True)

train_sdata_kido = os.path.join(train_sdata,'kido')
os.makedirs(train_sdata_kido, exist_ok=True)

valid_sdata_kido = os.path.join(valid_sdata,'kido')
os.makedirs(valid_sdata_kido, exist_ok=True)

test_sdata_kido = os.path.join(test_sdata,'kido')
os.makedirs(test_sdata_kido, exist_ok=True)

In [13]:
trainRatio = 0.6
validRatio = 0.2
testRatio = 1 - trainRatio - validRatio

In [14]:
numPath = os.path.join(os.getcwd(), 'kAug')
files = os.listdir(numPath)

train_num = int(trainRatio * len(files))
valid_num = int(validRatio * len(files))
test_num  = len(files) - train_num - valid_num

train_range = [0, train_num-1]
valid_range = [train_num, train_num + valid_num -1]
test_range  = [train_num + valid_num, train_num + valid_num + test_num-1]
print(f'train {train_range} valid {valid_range} test {test_range}')

train [0, 21] valid [22, 28] test [29, 37]


In [15]:
import os
import shutil
import random

# 데이터셋 디렉토리 경로 설정 (데이터셋이 저장된 경로로 수정하세요)
dataset_dir = os.path.join(os.getcwd(), 'kAug_multOverBatch')

# 새로운 train, valid, test 디렉토리 생성 경로
base_dir = os.path.join(os.getcwd(), 'kAug_multOverBatch_tvt')

# 클래스 목록
classes = ['kido']

# 폴더 경로 생성
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# 폴더 생성 함수
def create_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

# train, valid, test 폴더 및 각각의 클래스 폴더 생성
for cls in classes:
    create_dir(os.path.join(train_dir, cls))
    create_dir(os.path.join(valid_dir, cls))
    create_dir(os.path.join(test_dir, cls))

# 이미지 복사 함수
def copy_images(start_idx, end_idx, src_dir, dst_dir, label):
    srcFiles = os.listdir(src_dir)
    # print(f'be_sh {srcFiles}')
    srcFiles = random.sample(srcFiles, len(srcFiles))
    # print(f'af_sh {srcFiles}')
    for i in range(start_idx, end_idx + 1):
        file_name = f'{label}.{i}.jpg'
        # print(f' fileName {file_name}')
        src_path = os.path.join(src_dir, srcFiles[i])
        # print(f'src {src_path}')
        dst_path = os.path.join(dst_dir, file_name)
        # print(f'dst {dst_path}')
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_path)

# 클래스별로 train, valid, test 데이터셋 구성
for cls in classes:
    copy_images(train_range[0], train_range[1], os.path.join(dataset_dir), os.path.join(train_dir, cls), cls)
    # print(train_range[0], train_range[1], dataset_dir, os.path.join(train_dir, cls))

    # valid dataset 구성 (1000~1249)
    copy_images(valid_range[0], valid_range[1], os.path.join(dataset_dir), os.path.join(valid_dir, cls), cls)

    # test dataset 구성 (1250~1499)
    copy_images(test_range[0], test_range[1], os.path.join(dataset_dir), os.path.join(test_dir, cls), cls)

print("데이터셋 분할 및 복사가 완료되었습니다.")

데이터셋 분할 및 복사가 완료되었습니다.


In [16]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
results = model.train(data='data.yaml', epochs=50, imgsz=640)

engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train28, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False

train: Scanning C:\Users\SBA\yolo\yolov11\kDataset_yolo\train\labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\SBA\yolo\yolov11\kDataset_yolo\valid\labels.cache... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<?, ?it/s]


Plotting labels to runs\detect\train28\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train28
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

       8/50         0G     0.7157      1.813     0.9902         33        640: 100%|██████████| 2/2 [00:04<00:00,  2.02s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.80it/s]

                   all          7         14    0.00667          1      0.796      0.411



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/50         0G     0.6705      1.811      1.051         47        640:  50%|█████     | 1/2 [00:02<00:02,  2.74s/it]


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 4699140 bytes in function 'cv::OutOfMemoryError'


# 